# MNB Sentiment Scoring Model

<p> This is a process for training a Multinomial Naive Bayes sentiment classification model using tweets from a Kaggle competition

## Packages

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

## Import Training and Test Data

#### Our training data consists of the following attributes:
<ul> 
    <li><b>target:</b> the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)</li>
    <li><b>ids:</b> The id of the tweet ( 2087)</li>
    <li><b>date:</b> the date of the tweet (Sat May 16 23:58:44 UTC 2009)</li>
    <li><b>flag:</b> The query (lyx). If there is no query, then this value is NO_QUERY.</li>
    <li><b>user:</b> the user that tweeted (robotickilldozr)</li>
    <li><b>text:</b> the text of the tweet (Lyx is cool)</li>
</ul>

In [11]:
filename = "training_XL.csv"
data_set = pd.read_csv(filename, delimiter=',', encoding='ISO-8859-1', header=None)

## Process Data

In [12]:
data_set.columns = ["target","ids","date","flag","user","text"]
data_set.head()
y=data_set['target'].values
X=data_set['text'].values

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


## Prepare Data Holdout Test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(X_train[0])
print(y_train[0])
print(X_test[0])
print(y_test[0])